In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
from ipywidgets import interact
import numpy as np

from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [3]:
dt=1e-3
t=np.arange(0,5,dt)
target = np.array([0 if x < 1 else 5 for x in t]) # liters

p = figure(title="PID example", plot_height=300, plot_width=600, y_range=(-5,5))
tar = p.line(t, [0]*len(t), color="#2222aa", line_width=3, legend="Target level")
stat = p.line(t, [0]*len(t), color="#22FFaa", line_width=3, legend="Current level")
p.set(x_range=Range1d(0, 5), y_range=Range1d(-5, 10))
p.xaxis.axis_label="Time [s]"
p.xaxis.axis_label="Water level [l]"

In [4]:

def dx(x,u,A,B):
    return (A(x)+B(u))*dt

def y_new(x,u,C,D):
    return C(x)+D(u)

def u_pid(index,target,y,kp,ki,kd,control_period):
    e_p = target[index]-y[index-1] # instant error
    if(len(y)>2):
        e_i = np.mean([(t-yi) for t,yi in zip(target[:index-1],y)]) # history error
        e_d = ((target[index]-y[index-1])-(target[index-1]-y[index-control_period-1]))/(dt*control_period)
    else:
        e_d = target[index]-y[index-1]
        e_i = target[index]-y[index-1]
    up = kp*e_p
    ui = ki*e_i
    ud = kd*e_d
    return (up+ui+ud,up,ui,ud)


def state_matrix(x):
    if x>0:
        return -0.5 # constant small drain of 0.5 l/s
    else:
        return 0

def input_matrix(u):
    # flow rate can only take certain values
    return u

def output_matrix(x):
    return x # state directly observable

def feedthrough_matrix(u):
    return 0 # no direct feedthrough

def update(kp = 3, ki = 0,kd = 0,control_period=50):
    y0 = 3 # # start with some water in tup, 3 liters
    u0 = 0 # flow rate 0 at beginning
    x0 = y0
    dx0 = 0
    y_log = [y0]
    x_log = [x0]
    u_log = [u0]
    dx_log = [dx0]

    x=x0
    y=y0
    u=u0

    up_log =[0]
    ui_log =[0]
    ud_log =[0]

    for i in range(1,len(t)):
        if i==1 or (i%control_period)==0:
            u,up,ui,ud=u_pid(i,target,y_log,kp,ki,kd,control_period)
        dxt=dx(x,u,state_matrix,input_matrix)
        dx_log.append(dxt)
        x = x+dxt
        y = y_new(x,u,output_matrix,feedthrough_matrix)
        y_log.append(y)
        x_log.append(x)
        u_log.append(u)
        up_log.append(up)
        ui_log.append(ui)
        ud_log.append(ud)

    u_log=np.nan_to_num(np.array(u_log))
    y_log=np.nan_to_num(np.array(y_log))
    x_log=np.nan_to_num(np.array(x_log))
    dx_log=np.nan_to_num(np.array(dx_log))
    tar.data_source.data["y"] = target
    stat.data_source.data["y"] = y_log
    push_notebook()





In [5]:
h=show(p, notebook_handle=True)


In [6]:
i=interact(update, kp=(0,100,0.1), ki=(0,100,0.1), kd=(0, 100,0.1),control_period=(1,100))
